In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random
from torch.utils.tensorboard import SummaryWriter


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/'Colab Notebooks'/'yai_9기 겨울방학'/ToyProject

/content/drive/MyDrive/Colab Notebooks/yai_9기 겨울방학/ToyProject


In [4]:
writer = SummaryWriter(log_dir = '/run/experiment_1')

In [5]:
csv_path = "./final.csv"
image_path = "./data/images"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [6]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [7]:
class AllDataset(Dataset):
    def __init__(self, csv_path, image_path, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(AllDataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        self.images = []
        for Id in csv['Id']:
            Id = str(Id)
            image = Image.open(image_path+'/'+Id+'.jpg')
            self.images.append(image_transform(image))
            image.close()
        
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(list(map(lambda x: (x - np.array([0.61756694, 5.49324267, 4.76044624, 3.05992563])) / np.array([0.39076653, 1.28855836, 0.77676551, 0.51616518]), csv_np[:,[2,9,10,11]])), dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        return self.images[idx], self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [8]:
data_length = len(pd.read_csv(csv_path))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

image_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5),
                                                         (0.5, 0.5, 0.5))
                                     ])

train_dataset = AllDataset(csv_path, image_path, image_transform)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet18.fc = nn.Sequential(nn.Linear(512, 2048),
                                         nn.ReLU(),
                                        nn.Linear(2048, 2048),
                                        nn.ReLU())
        self.resenet18 = self.resnet18.to(device)
        
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.csvfc1 = nn.Linear(220, 4096)
        self.csvfc2 = nn.Linear(4096, 4096)
        self.csvfc3 = nn.Linear(4096, 2048)
        
        self.fc1 = nn.Linear(4096, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1)
        self.dropout = nn.Dropout()
    
    def forward(self, image, x, y):
        image = self.resnet18(image)
        
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.fc(y)
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        
        x = self.dropout(self.act(self.csvfc1(x)))
        x = self.dropout(self.act(self.csvfc2(x)))
        x = self.dropout(self.act(self.csvfc3(x)))
        x = torch.cat((x, image), dim=1)
        
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        return self.fc3(x)

In [13]:
model = Net().to(device)

epochs = 100
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for Image, X1, X2, Y in train_loader:
        Image = Image.to(device)
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(Image, X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for Image_val, X1_val, X2_val, Y_val in val_loader:
            Image_val = Image_val.to(device)
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(Image_val, X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 3954.2812, val_loss: 4555.2041
[Epoch:  2] Average loss: 2495.0620, val_loss: 1986.6316
[Epoch:  3] Average loss: 2147.5293, val_loss: 2500.3796
[Epoch:  4] Average loss: 1884.9471, val_loss: 1747.1145
[Epoch:  5] Average loss: 1613.8328, val_loss: 1487.1334
[Epoch:  6] Average loss: 1461.0271, val_loss: 2078.4705
[Epoch:  7] Average loss: 1478.5674, val_loss: 1544.0852
[Epoch:  8] Average loss: 1569.2729, val_loss: 1493.1931
[Epoch:  9] Average loss: 1148.6943, val_loss: 2343.4612
[Epoch: 10] Average loss: 1676.8287, val_loss: 2001.4698
[Epoch: 11] Average loss: 1768.6604, val_loss: 1606.4700
[Epoch: 12] Average loss: 1365.5851, val_loss: 1533.1925
[Epoch: 13] Average loss: 1220.2422, val_loss: 1249.0044
[Epoch: 14] Average loss: 1150.4320, val_loss: 1772.9288
[Epoch: 15] Average loss: 1289.6927, val_loss: 1284.5366
[Epoch: 16] Average loss: 1202.6510, val_loss: 1620.6465
[Epoch: 17] Average loss: 1299.9739, val_loss: 1476.9993
[Epoch: 18] Average loss: 954.3

In [15]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
test_loader = DataLoader(test_dataset, batch_size=1, shuffle = True)
pred = []
label = []
with torch.no_grad():
    for Image_test, X1_test, X2_test, Y_test in test_loader:
        Image_test =Image_test.to(device)
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(Image_test, X1_test, X2_test)
        print(f"결과값, 예측값 : {Y_test}, {prediction}")
        pred.append(prediction)
        label.append(Y_test)
    pred = torch.tensor(pred)
    label = torch.tensor(label)
    mean = torch.mean(label)
    ss_tot += torch.sum((label - mean) ** 2)
    ss_res += torch.sum((label - pred) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    MSE = (torch.sum((label - pred)**2))**(1/2) / len(pred.numpy())
    print(f"Accuracy: {accuracy*100:.2f}   MSE : {MSE}", )

결과값, 예측값 : tensor([[1060.5601]], device='cuda:0'), tensor([[1311.4197]], device='cuda:0')
결과값, 예측값 : tensor([[1931.9200]], device='cuda:0'), tensor([[2138.1453]], device='cuda:0')
결과값, 예측값 : tensor([[2111.8201]], device='cuda:0'), tensor([[2450.6060]], device='cuda:0')
결과값, 예측값 : tensor([[2077.8899]], device='cuda:0'), tensor([[2291.7283]], device='cuda:0')
결과값, 예측값 : tensor([[2425.6101]], device='cuda:0'), tensor([[3729.2605]], device='cuda:0')
결과값, 예측값 : tensor([[1936.0601]], device='cuda:0'), tensor([[1778.2296]], device='cuda:0')
결과값, 예측값 : tensor([[1408.6500]], device='cuda:0'), tensor([[1405.6823]], device='cuda:0')
결과값, 예측값 : tensor([[971.7500]], device='cuda:0'), tensor([[1343.0707]], device='cuda:0')
결과값, 예측값 : tensor([[1301.1100]], device='cuda:0'), tensor([[1403.6890]], device='cuda:0')
결과값, 예측값 : tensor([[2359.8101]], device='cuda:0'), tensor([[3091.6699]], device='cuda:0')
결과값, 예측값 : tensor([[1578.4600]], device='cuda:0'), tensor([[1880.0687]], device='cuda:0')
결과값, 예측값 : 

In [17]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
test_loader = DataLoader(test_dataset, batch_size=1, shuffle = True)
pred = []
label = []
with torch.no_grad():
    for Image_test, X1_test, X2_test, Y_test in test_loader:
        if Y_test < 15000:
            continue
        Image_test =Image_test.to(device)
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(Image_test, X1_test, X2_test)
        print(f"결과값, 예측값 : {Y_test}, {prediction}")
        pred.append(prediction)
        label.append(Y_test)
    pred = torch.tensor(pred)
    label = torch.tensor(label)
    mean = torch.mean(label)
    ss_tot += torch.sum((label - mean) ** 2)
    ss_res += torch.sum((label - pred) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    MSE = (torch.sum((label - pred)**2))**(1/2) / len(pred.numpy())
    print(f"Accuracy: {accuracy*100:.2f}   MSE : {MSE}", )

결과값, 예측값 : tensor([[18073.7402]], device='cuda:0'), tensor([[22724.8223]], device='cuda:0')
결과값, 예측값 : tensor([[64733.5195]], device='cuda:0'), tensor([[74729.9766]], device='cuda:0')
결과값, 예측값 : tensor([[18681.6406]], device='cuda:0'), tensor([[20098.4980]], device='cuda:0')
결과값, 예측값 : tensor([[15690.7100]], device='cuda:0'), tensor([[16319.7178]], device='cuda:0')
결과값, 예측값 : tensor([[15504.2900]], device='cuda:0'), tensor([[15998.9873]], device='cuda:0')
결과값, 예측값 : tensor([[68738.5234]], device='cuda:0'), tensor([[57513.7070]], device='cuda:0')
결과값, 예측값 : tensor([[18054.6504]], device='cuda:0'), tensor([[47179.7227]], device='cuda:0')
결과값, 예측값 : tensor([[16247.8701]], device='cuda:0'), tensor([[18057.4062]], device='cuda:0')
결과값, 예측값 : tensor([[41056.6484]], device='cuda:0'), tensor([[34432.0625]], device='cuda:0')
결과값, 예측값 : tensor([[19164.7695]], device='cuda:0'), tensor([[25296.5527]], device='cuda:0')
결과값, 예측값 : tensor([[16186.7305]], device='cuda:0'), tensor([[15355.3320]], devic

In [18]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
test_loader = DataLoader(test_dataset, batch_size=1, shuffle = True)
pred = []
label = []
with torch.no_grad():
    for Image_test, X1_test, X2_test, Y_test in test_loader:
        if Y_test < 20000:
            continue
        Image_test =Image_test.to(device)
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(Image_test, X1_test, X2_test)
        print(f"결과값, 예측값 : {Y_test}, {prediction}")
        pred.append(prediction)
        label.append(Y_test)
    pred = torch.tensor(pred)
    label = torch.tensor(label)
    mean = torch.mean(label)
    ss_tot += torch.sum((label - mean) ** 2)
    ss_res += torch.sum((label - pred) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    MSE = (torch.sum((label - pred)**2))**(1/2) / len(pred.numpy())
    print(f"Accuracy: {accuracy*100:.2f}   MSE : {MSE}", )

결과값, 예측값 : tensor([[64733.5195]], device='cuda:0'), tensor([[74729.9766]], device='cuda:0')
결과값, 예측값 : tensor([[41056.6484]], device='cuda:0'), tensor([[34432.0625]], device='cuda:0')
결과값, 예측값 : tensor([[68738.5234]], device='cuda:0'), tensor([[57513.7070]], device='cuda:0')
Accuracy: 39.73   MSE : 5475.30615234375


전체 price : 89.49 % MSE:40.74   
price 10000 이상 데이터만 사용 : 77.00% MSE : 1676     
price 15000 이상 데이터만 사용 : 71.30% MSE : 3128  
price 20000 이상 데이터만 사용 : 39.73% MSE : 5475.30  
하지만 eda 보시면 10000~ 20000도 정상적인 범주로 들어가서
만약 price가 30000 이상인 데이터만 또 다루면 metric이 떨어질 것으로 예상
epoch는 동일하게 두고 비교한 것입니다.